In [15]:
import os
txt_files = glob.glob("shakespeare-db/*.txt")
print(txt_files)

["shakespeare-db\\All's Well That Ends Well.txt", 'shakespeare-db\\Antony and Cleopatra.txt', 'shakespeare-db\\As You Like It.txt', 'shakespeare-db\\Comedy of Errors.txt', 'shakespeare-db\\Coriolanus.txt', 'shakespeare-db\\Cymbeline.txt', 'shakespeare-db\\Hamlet.txt', 'shakespeare-db\\Henry IV, part 1.txt', 'shakespeare-db\\Henry IV, part 2.txt', 'shakespeare-db\\Henry V.txt', 'shakespeare-db\\Henry VI, part 1.txt', 'shakespeare-db\\Henry VI, part 2.txt', 'shakespeare-db\\Henry VI, part 3.txt', 'shakespeare-db\\Henry VIII.txt', 'shakespeare-db\\Julius Caesar.txt', 'shakespeare-db\\King John.txt', 'shakespeare-db\\King Lear.txt', "shakespeare-db\\Love's Labour's Lost.txt", 'shakespeare-db\\Macbeth.txt', 'shakespeare-db\\Measure for Measure.txt', 'shakespeare-db\\Merchant of Venice.txt', 'shakespeare-db\\Merry Wives of Windsor.txt', "shakespeare-db\\Midsummer Night's Dream.txt", 'shakespeare-db\\Much Ado About Nothing.txt', 'shakespeare-db\\Othello.txt', 'shakespeare-db\\Pericles.txt', '

In [16]:
text = ""

for name in txt_files:
    with open(txt_files[0],'rt') as fd:
        st = fd.readlines()
        text+=(" ".join(st))

In [21]:
print(text[:1000])


 
 
 
                            all's well that ends well
 shakespeare_homepage | all's_well_that_ends_well | entire play
 **** act i ****
 **** scene i. rousillon. the count's palace. ****
      enter bertram, the countess of rousillon, helena, and lafeu, all in
      black
 countess
      in delivering my son from me, i bury a second husband.
 bertram
      and i in going, madam, weep o'er my father's death
      anew: but i must attend his majesty's command, to
      whom i am now in ward, evermore in subjection.
 lafeu
      you shall find of the king a husband, madam; you,
      sir, a father: he that so generally is at all times
      good must of necessity hold his virtue to you; whose
      worthiness would stir it up where it wanted rather
      than lack it where there is such abundance.
 countess
      what hope is there of his majesty's amendment?
 lafeu
      he hath abandoned his physicians, madam; under whose
      practises he hath persecuted time with hope, and
    

In [20]:
print('Corpus Length:',len(text))
text = text.lower()

Corpus Length: 5380776


In [19]:
from __future__ import print_function
import keras
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
import numpy as np
import random
import sys
import io

Using TensorFlow backend.
C:\Users\HP\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\HP\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\HP\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\HP\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: Future

In [22]:
chars = sorted(list(set(text)))
print('Total Chars: ',len(chars))
char_indices = dict((c,i) for i,c in enumerate(chars))
indices_char = dict((i,c) for i,c in enumerate(chars))

Total Chars:  42


In [25]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0,len(text)-maxlen,step):
    sentences.append(text[i:i+maxlen])
    next_chars.append(text[i+maxlen])
print('no. sequences:', len(sentences))

no. sequences: 1793579


In [27]:
x = np.zeros((len(sentences),maxlen,len(chars)),dtype=np.bool)
y = np.zeros((len(sentences),len(chars)),dtype=np.bool)
for i,sentence in enumerate(sentences):
    for t,char in enumerate(sentence):
        x[i,t,char_indices[char]] = 1
    y[i,char_indices[next_chars[i]]] = 1

In [29]:
###Building the Model###

In [32]:
print("Build Model..")
model = Sequential()
model.add(LSTM(128,input_shape=(maxlen,len(chars))))
model.add(Dense(len(chars),activation='softmax'))

optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',optimizer=optimizer)

Build Model..
Instructions for updating:
Colocations handled automatically by placer.


In [33]:
def sample(preds,temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)/temperature
    exp_preds = np.exp(preds)
    preds = exp_preds/np.sum(exp_preds) #for normalizing
    pb = np.random.multinomial(1,preds,1) #for selecting a depending based on probability
    return np.argmax(pb)

In [37]:
##https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/LambdaCallback##

In [38]:
#for LambdaCallback

def on_epoch_end(epoch,_):
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
    

In [39]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [41]:
model.fit(x, y,
          batch_size=128,
          epochs=2,
          callbacks=[print_callback])

Epoch 1/2
1793579/1793579 [==============================] - 2190s 1ms/step - loss: 0.9007

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "t the white death sit on thy cheek for e"
t the white death sit on thy cheek for ence her parolles
      more than in he more to praroour marit?
 first lord
      i have no our i wore mers, in that her parter.
 helena
      if her head.
 paro

C:\Users\HP\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


lles
      i have no hir beeding print thou art me
      in that i have no hir beed intonted the
      wisle me sense as more to prarost you, and that he hourselves:
      i am friend brood marr; and the sake the friendly her hearts;
      
----- diversity: 0.5
----- Generating with seed: "t the white death sit on thy cheek for e"
t the white death sit on thy cheek for ence
      flest that at stears erous contreat of
      if i had ham hee it do you: there's no here;
      that well despeat her procher do mend, sir; i have and
      his lost the plamer the fiend the paroin to this
      more to scarr; i nate in here is not and my repeated
      and stell the shall be bother her part to her.
 helena
      where in me and care strefent of stratt
      in the reper
----- diversity: 1.0
----- Generating with seed: "t the white death sit on thy cheek for e"
t the white death sit on thy cheek for elt
      than thou tamaning on't he has dead it?  fine.
 clown
      you shoth less thee.
 ber